In [26]:
import requests
import json

In [ ]:
df.read_csv('../clean_categorizated_data_without_outliers_with_coordinates.csv', sep=';')

In [38]:
all_categories = {
    "eat":{
        "amenities": ["cafe", "fast_food", "pub", "restaurant"],
        "data": set()
    },
    "edu":{
        "amenities": ["college", "kindergarten", "library", "school", "university"],
        "data": set()
    },
    "bank":{
        "amenities": ["bank", "atm"],
        "data": set()
    },
    "health":{
        "amenities": ["clinic", "hospital", "pharmacy"],
        "data": set()
    },
    "culture":{
        "amenities": ["cinema", "fountain", "theatre"],
        "data": set()
    },
}

In [32]:
overpass_url = "http://overpass-api.de/api/interpreter"

for category in all_categories:
    for amenity in all_categories[category]['amenities']:
        overpass_query = f"""
        [out:json];
        node(57.5,55,58.5,56.5)[amenity={amenity}];out;
        """
        response = requests.get(overpass_url, 
                                params={'data': overpass_query})
        try:
            data = response.json()
            for element in data['elements']:
              if element['type'] == 'node':
                lon = element['lon']
                lat = element['lat']
                all_categories[category]['data'].add((lat, lon))
        except Exception as ex:
            print(amenity)
            print(ex)

planetarium
Expecting value: line 1 column 1 (char 0)
bench
Expecting value: line 1 column 1 (char 0)
shelter
Expecting value: line 1 column 1 (char 0)


In [40]:
for key in all_categories:
    print(key + ": " + str(len(all_categories[key]['data'])))

eat: 0
edu: 0
bank: 0
health: 0
culture: 0


In [39]:
all_categories

{'eat': {'amenities': ['cafe', 'fast_food', 'pub', 'restaurant'],
  'data': set()},
 'edu': {'amenities': ['college',
   'kindergarten',
   'library',
   'school',
   'university'],
  'data': set()},
 'bank': {'amenities': ['bank', 'atm'], 'data': set()},
 'health': {'amenities': ['clinic', 'hospital', 'pharmacy'], 'data': set()},
 'culture': {'amenities': ['cinema', 'fountain', 'theatre'], 'data': set()}}